In [ ]:
#This notebook trains a GCN autoencoder network first before combining the encoder network with another FF network 
#to predict single cell identities

In [1]:
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter
import torch
from torch.autograd import Variable
from torch.utils.data import TensorDataset,DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torch_geometric.utils import get_laplacian
import pandas as pd
import numpy as np
import math
import networkx as nx
import matplotlib
from sklearn.preprocessing import OneHotEncoder

In [2]:
#Loading adjacency matrix and filtered gene expression

adj_mat = np.load('data/adj_mat_on_normalized2.npy')
pbmc = np.load('data/pbmc_normalized_filtered2.npy')

In [3]:
N = adj_mat.shape[0] # number of nodes in a graph
D = np.sum(adj_mat, 0) # node degrees
D_hat_pos = np.diag((D + 1e-5)**(0.5)) # normalized node degrees
D_hat_neg = np.diag((D + 1e-5)**(-0.5)) # normalized node degrees
L = np.identity(N) + D_hat_neg.dot(adj_mat).dot(D_hat_pos) # Laplacian

In [4]:
# Spectral convolution on graphs
# X is an N×1 matrix of 1-dimensional node features
# L is an N×N graph Laplacian computed above
# W_spectral are N×F weights (filters) that we want to train
from scipy.sparse.linalg import eigsh # assumes L to be symmetric

Λ,V = eigsh(L,k=20,which='SM') # eigen-decomposition (i.e. find Λ,V)


In [5]:
V.T.shape

(20, 960)

In [6]:
#Autoencoder model

class GraphConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    
    def __init__(self, input_dim, in_features, out_features, batch_size, bias=False):
        super(GraphConvolution, self).__init__()
        self.input_dim = input_dim
        self.batch_size = batch_size
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(torch.FloatTensor(self.batch_size, self.in_features, self.out_features), requires_grad=True)
        if bias:
            self.bias = Parameter(torch.FloatTensor(self.batch_size, self.input_dim, self.out_features), requires_grad=True)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight, gain=nn.init.calculate_gain('relu'))
#         stdv = 1. / math.sqrt(self.weight.size(1))
#         self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            torch.nn.init.xavier_uniform_(self.bias, gain=nn.init.calculate_gain('relu'))

    def forward(self, input, adj):
#         input = torch.permute(input, (0, 2, 1))
        support = torch.bmm(input, self.weight)
        output = torch.matmul(adj, support)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
    
class autoencoder(nn.Module):
    
    def __init__(self, input_dim, nhidden, p, adj, batch_size, dropout):
        super(autoencoder, self).__init__()
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.nhidden = nhidden
        self.dropout = nn.Dropout(dropout)
        self.poolsize = 8
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.adj = Parameter(torch.from_numpy(adj).contiguous().float(), requires_grad=False).to(self.device)
        self.p = p
        self.gcn = GraphConvolution(self.input_dim, 1, self.nhidden, self.batch_size, bias=False)
        self.gcn2 = GraphConvolution(self.input_dim, self.nhidden, self.nhidden//2, self.batch_size, bias=False)
        self.encoder_a1 = nn.Linear(int(((self.input_dim//self.p)*nhidden)), 128, bias=True)
        self.encoder_a2 = nn.Linear(128, 32, bias=True)
        
        self.decoder1 = nn.Linear(32, 128, bias=True)
        self.decoder2 = nn.Linear(128, self.input_dim, bias=True)
        
        #Make sure gene order in adj is same!
        #how to implement the attention layer?
        #what's the diff between Parameter and Variable
        
    def graph_max_pool(self, x, p):
        if p > 1:
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            return x
        else:
            return x
        
    def forward(self, x):
        #encoder
        stream1 = torch.unsqueeze(x, -1)
        stream1 = F.relu(self.gcn(stream1, self.adj))
#         stream1 = F.relu(self.gcn2(stream1, self.adj))
        stream1 = self.graph_max_pool(stream1, self.p)
        stream1 = torch.flatten(stream1, start_dim = 1)
        stream1 = F.relu(self.encoder_a1(stream1))
        stream1 = F.relu(self.encoder_a2(stream1))
        
#         stream2 = F.relu(self.encoder_b1(x))
#         stream2 = F.relu(self.encoder_b2(stream2))
        
#         combined = torch.cat((stream1, stream2), -1)
        
        #decoder
        stream1 = F.relu(self.decoder1(stream1))
        outputs = F.relu(self.decoder2(stream1))
        
        return outputs

In [7]:
#Autoencoder using attention

class GraphAttConvolution(nn.Module):
    """
    Simple GCN layer, similar to https://arxiv.org/abs/1609.02907
    """
    
    def __init__(self, input_dim, in_features, out_features, batch_size, bias=False):
        super(GraphAttConvolution, self).__init__()
        self.input_dim = input_dim
        self.batch_size = batch_size
        self.in_features = in_features
        self.out_features = out_features
        self.att = Parameter(torch.FloatTensor(self.input_dim, self.input_dim), requires_grad=True)
        self.weight = Parameter(torch.FloatTensor(self.in_features, self.out_features), requires_grad=True)
#         self.att = Parameter(torch.FloatTensor(self.input_dim, self.out_features), requires_grad=False)
        if bias:
            self.bias = Parameter(torch.FloatTensor(self.input_dim, self.out_features), requires_grad=True)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight, gain=nn.init.calculate_gain('relu'))
        torch.nn.init.xavier_uniform_(self.att, gain=nn.init.calculate_gain('relu'))
#         stdv = 1. / math.sqrt(self.weight.size(1))
#         self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            torch.nn.init.xavier_uniform_(self.bias, gain=nn.init.calculate_gain('relu'))

    def forward(self, input, adj):
#         input = torch.permute(input, (0, 2, 1))
        self.att = Parameter(torch.softmax(self.att, dim=1), requires_grad=True)
        adj = torch.matmul(self.att, adj)
        support = torch.matmul(input, self.weight)
        output = torch.matmul(adj, support)
#         att = torch.softmax(output, dim=1)
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.in_features) + ' -> ' \
               + str(self.out_features) + ')'
    
class autoencoder_att(nn.Module):
    
    def __init__(self, input_dim, nhidden, p, adj, batch_size, dropout):
        super(autoencoder_att, self).__init__()
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.nhidden = nhidden
#         self.fc1 = nn.Linear(input_dim, nhidden, bias=False)
        self.dropout = nn.Dropout(dropout)
        self.poolsize = 8
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.adj = Parameter(torch.from_numpy(adj).contiguous().float(), requires_grad=False).to(self.device)
        self.p = p
        self.gcn = GraphAttConvolution(self.input_dim, 1, self.nhidden, self.batch_size, bias=False)
#         self.gcn2 = GraphAttConvolution(self.input_dim, self.nhidden, self.nhidden//2, self.batch_size, bias=False)
        self.encoder_a1 = nn.Linear(int(((self.input_dim//self.p)*nhidden)), 128, bias=True)
        self.encoder_a2 = nn.Linear(128, 32, bias=True)
        
        self.decoder1 = nn.Linear(32, 128, bias=True)
        self.decoder2 = nn.Linear(128, self.input_dim, bias=True)
        
        
        #Make sure gene order in adj is same!
        #how to implement the attention layer?
        #what's the diff between Parameter and Variable
        
    def graph_max_pool(self, x, p):
        if p > 1:
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            return x
        else:
            return x
        
    def forward(self, x):
        #encoder
        stream1 = torch.unsqueeze(x, -1)
        stream1 = F.relu(self.gcn(stream1, self.adj))
#         stream1 = F.relu(self.gcn2(stream1, self.adj))
        stream1 = self.graph_max_pool(stream1, self.p)
        stream1 = torch.flatten(stream1, start_dim = 1)
        stream1 = F.relu(self.encoder_a1(stream1))
        stream1 = F.relu(self.encoder_a2(stream1))
        
        #decoder
        stream1 = F.relu(self.decoder1(stream1))
        outputs = F.relu(self.decoder2(stream1))
        
        return outputs

In [8]:
#Autoencoder using spectral theory model

class SpectralConvolution(nn.Module):
    
    def __init__(self, input_dim, out_features, batch_size, bias=False):
        super(SpectralConvolution, self).__init__()
        self.input_dim = input_dim
        self.batch_size = batch_size
        self.out_features = out_features
        self.weight_spectral = Parameter(torch.FloatTensor(self.input_dim, self.out_features), requires_grad=True)
        if bias:
            self.bias = Parameter(torch.FloatTensor(self.batch_size, self.input_dim, self.out_features), requires_grad=True)
        else:
            self.register_parameter('bias', None)
        self.reset_parameters()
        

    def reset_parameters(self):
        torch.nn.init.xavier_uniform_(self.weight_spectral, gain=nn.init.calculate_gain('relu'))
#         stdv = 1. / math.sqrt(self.weight_spectral.size(1))
#         self.weight_spectral.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            torch.nn.init.xavier_uniform_(self.bias, gain=nn.init.calculate_gain('relu'))

    def forward(self, input, V):
        X_hat = torch.matmul(V.T, input) # 20×1 node features in the "spectral" domain
        W_hat = torch.matmul(V.T, self.weight_spectral) # 20×F filters in the "spectral" domain
        output = torch.matmul(V, X_hat * W_hat)  # N×F result of convolution
        if self.bias is not None:
            return output + self.bias
        else:
            return output

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
               + str(self.out_features) + ')'
    
class spectral_autoencoder(nn.Module):
    
    def __init__(self, input_dim, nhidden, p, adj, batch_size, dropout):
        super(spectral_autoencoder, self).__init__()
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.nhidden = nhidden
        self.dropout = nn.Dropout(dropout)
        self.poolsize = 8
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.adj = Parameter(torch.from_numpy(adj).contiguous().float(), requires_grad=False).to(self.device)
        self.p = p
        self.gcn = SpectralConvolution(self.input_dim, self.nhidden, self.batch_size, bias=False)
        self.gcn2 = SpectralConvolution(self.input_dim, self.nhidden, self.batch_size, bias=False)
        self.encoder_a1 = nn.Linear(int(((self.input_dim//self.p)*nhidden)), 128, bias=True)
        self.encoder_a2 = nn.Linear(128, 32, bias=True)
        
        self.decoder1 = nn.Linear(32, 128, bias=True)
        self.decoder2 = nn.Linear(128, self.input_dim, bias=True)
        
        #Make sure gene order in adj is same!
        #how to implement the attention layer?
        #what's the diff between Parameter and Variable
        
    def graph_max_pool(self, x, p):
        if p > 1:
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            return x
        else:
            return x
        
    def forward(self, x):
        #encoder
        stream1 = torch.unsqueeze(x, -1)
        stream1 = F.relu(self.gcn(stream1, self.adj))
        stream1 = F.relu(self.gcn2(stream1, self.adj))
        stream1 = self.graph_max_pool(stream1, self.p)
        stream1 = torch.flatten(stream1, start_dim = 1)
        stream1 = F.relu(self.encoder_a1(stream1))
        stream1 = self.dropout(stream1)
        stream1 = F.relu(self.encoder_a2(stream1))
        
        #decoder
        stream1 = F.relu(self.decoder1(stream1))
        stream1 = self.dropout(stream1)
        outputs = F.relu(self.decoder2(stream1))
        
        return outputs

In [9]:
class simpleAE(nn.Module):
    def __init__(self,input_dim,device):
        super(simpleAE, self).__init__()
        self.fc_input=nn.Linear(input_dim,128)
        self.fc1=nn.Linear(128,32)
        self.fc2=nn.Linear(32,128)
        self.fc3=nn.Linear(128,input_dim)
        self.dropout=nn.Dropout(0.3)
        self.device=device
    
    def forward(self,x):
        x.to(self.device)
        x=F.relu(self.fc_input(x))
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        outputs=F.relu(self.fc3(x))
        return outputs

In [14]:
#GCN AE

batch_size = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = spectral_autoencoder(pbmc.shape[1], nhidden=32, p=8, adj=V, batch_size=batch_size, dropout=0.3)
# print(model)

Using cpu device


In [12]:
for k,v in model.state_dict().items():
    print(k)

adj
gcn.weight
gcn2.weight
encoder_a1.weight
encoder_a1.bias
encoder_a2.weight
encoder_a2.bias
decoder1.weight
decoder1.bias
decoder2.weight
decoder2.bias


In [10]:
#SimpleAE

batch_size = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

model = simpleAE(pbmc.shape[1], device=device)
# print(model)

Using cpu device


In [15]:
loss_fn = torch.nn.MSELoss()
# optimizer = torch.optim.Adam(model.parameters(),
#                              lr = 1e-2,
#                              weight_decay = 1e-8)

optimizer = torch.optim.SGD(model.parameters(),
                             lr = 1e-3)

# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50000, gamma=0.1)

x_tensor = torch.tensor(pbmc[:len(pbmc)].astype(np.float32))

dataset = TensorDataset(x_tensor, x_tensor)

# batch_size = 32
validation_split = .2
shuffle_dataset = True
random_seed= 30

# Creating data indices for training and validation splits:
dataset_size = len(x_tensor)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [16]:
def train(dataloader, model, loss_fn, optimizer, scheduler=None):
    size = math.floor(len(dataloader.dataset)*0.8)
    model.train()
    correct = 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error 
        pred = model(X)
        loss = loss_fn(pred, y)

        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
#         scheduler.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            correct /= 100*batch_size

            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            correct = 0
            
def test(dataloader, model, loss_fn):
    size = math.floor(len(dataloader.dataset)*0.2)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Avg loss: {test_loss:>8f} \n")

In [182]:
pred=model(torch.tensor(pbmc[0:100].astype(np.float32)))

In [230]:
for X, y in validation_loader:
    X, y = X.to(device), y.to(device)
    pred = model(X)
    print(pred[10][0:10])
    print(y[10][0:10])
    break

tensor([ 99.5927, 165.9684,  83.0343, 124.8530,  74.8347, 116.0187,  33.0841,
        157.9944, 124.0977, 141.3166], grad_fn=<SliceBackward0>)
tensor([ 95.3137, 158.8562,  79.4281, 119.1422,  71.4853, 111.1994,  31.7712,
        150.9134, 119.1422, 135.0278])


In [17]:
epochs = 50
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(validation_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 561.179932  [    0/52754]
loss: 560.909912  [ 3200/52754]
loss: 560.664856  [ 6400/52754]
loss: 560.305847  [ 9600/52754]
loss: 559.792664  [12800/52754]
loss: 559.222839  [16000/52754]
loss: 558.446228  [19200/52754]
loss: 557.412170  [22400/52754]
loss: 555.661682  [25600/52754]
loss: 553.024353  [28800/52754]
loss: 546.533813  [32000/52754]
loss: 525.017700  [35200/52754]
loss: 65.399437  [38400/52754]
loss: 56.758110  [41600/52754]
loss: 50.562012  [44800/52754]
loss: 42.217869  [48000/52754]
loss: 46.292110  [51200/52754]
Avg loss: 26.590253 

Epoch 2
-------------------------------
loss: 40.527897  [    0/52754]
loss: 30.902571  [ 3200/52754]
loss: 36.361240  [ 6400/52754]
loss: 33.271950  [ 9600/52754]
loss: 31.900114  [12800/52754]
loss: 30.193987  [16000/52754]
loss: 34.729111  [19200/52754]
loss: 31.220364  [22400/52754]
loss: 29.825640  [25600/52754]
loss: 25.102697  [28800/52754]
loss: 31.766535  [32000/52754]
loss: 26.691778  [

In [78]:
#saving V

np.save('data/best_eigenvectors.npy', V)

In [ ]:
# Print model's state_dict
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# # Print optimizer's state_dict
# print("Optimizer's state_dict:")
# for var_name in optimizer.state_dict():
#     print(var_name, "\t", optimizer.state_dict()[var_name])

In [83]:
# torch.save(model.state_dict(), 'data/autoencoder')

torch.save({
            'epoch': epochs,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict()
            }, 'data/autoencoder_best_spectralx2')

In [23]:
trained_model_dict = model.state_dict()

In [ ]:
G = nx.from_numpy_matrix(adj_mat)
nx.draw_networkx(G, node_size=20, with_labels=False, linewidths=0.2)

In [18]:
labels = pd.read_csv('data/Intra-dataset/Zheng 68K/Labels.csv')

In [19]:
encoder = OneHotEncoder(handle_unknown='ignore')
labels_df = pd.DataFrame(encoder.fit_transform(labels[['x']]).toarray())

print(labels_df)

        0    1    2    3    4    5    6    7    8    9    10
0      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
1      0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
2      0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0
3      0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0
4      0.0  0.0  0.0  0.0  1.0  0.0  0.0  0.0  0.0  0.0  0.0
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...
65938  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
65939  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
65940  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0
65941  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0
65942  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  1.0  0.0  0.0

[65943 rows x 11 columns]


In [ ]:
trained_model_dict = torch.load('data/autoencoder')

In [24]:
#skeleton for loading trained spectral encoder

class trained_encoder(nn.Module):
    
    def __init__(self, input_dim, nhidden, p, adj, batch_size, dropout):
        super(trained_encoder, self).__init__()
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.nhidden = nhidden
        self.dropout = nn.Dropout(dropout)
        self.poolsize = 8
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.p = p
        self.adj = Parameter(torch.from_numpy(adj).contiguous().float(), requires_grad=False).to(self.device)
        self.gcn = SpectralConvolution(self.input_dim, self.nhidden, self.batch_size, bias=False)
        self.gcn2 = SpectralConvolution(self.input_dim, self.nhidden, self.batch_size, bias=False)
        self.encoder_a1 = nn.Linear(int(((self.input_dim//self.p)*nhidden)), 128, bias=True)
        self.encoder_a2 = nn.Linear(128, 32, bias=True)
        
#         self.decoder1 = nn.Linear(32, 128, bias=True)
        

    def graph_max_pool(self, x, p):
        if p > 1:
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            return x
        else:
            return x
        
    def forward(self, x):
        
        #encoder
        stream1 = torch.unsqueeze(x, -1)
        stream1 = F.relu(self.gcn(stream1, self.adj))
        stream1 = F.relu(self.gcn2(stream1, self.adj))
        stream1 = self.graph_max_pool(stream1, self.p)
        stream1 = torch.flatten(stream1, start_dim = 1)
        stream1 = F.relu(self.encoder_a1(stream1))
        stream1 = self.dropout(stream1)
        outputs = F.relu(self.encoder_a2(stream1))
        
#         outputs = F.relu(self.decoder1(stream1))
        
        return outputs

In [25]:
#Trained encoder

batch_size = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

trained_encoder_model = trained_encoder(pbmc.shape[1], nhidden=32, p=8, adj=V, batch_size=batch_size, dropout=0.3)
# print(model)

Using cpu device


In [26]:
trained_encoder_model_dict = trained_encoder_model.state_dict()

# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in trained_model_dict.items() if k in trained_encoder_model_dict}
# 2. overwrite entries in the existing state dict
trained_encoder_model_dict.update(pretrained_dict) 
# # 3. load the new state dict
trained_encoder_model.load_state_dict(pretrained_dict)

<All keys matched successfully>

In [ ]:
for name, param in trained_encoder_model.state_dict().items():
    print(name, param)

In [211]:
# grad_lst = ['encoder_a1.weight', 'encoder_a2.bias']

# for name, param in trained_encoder_model.named_parameters():
#     if name not in grad_lst:
#         param.requires_grad = False

In [212]:
# for name, param in trained_encoder_model.named_parameters():
#     if param.requires_grad:
#         print(name)

encoder_a1.weight
encoder_a2.bias


In [27]:
for param in trained_encoder_model.parameters():
    param.requires_grad = False

In [28]:
class combined_net(nn.Module):
    def __init__(self, encoder, input_dim, device):
        super(combined_net, self).__init__()
        self.encoder = encoder
        self.fc_input=nn.Linear(input_dim,512)
        self.fc1=nn.Linear(512,128)
        self.fc2=nn.Linear(128,32)
        self.dropout=nn.Dropout(0.3)
        self.fc3=nn.Linear(64, 11)
        self.device=device
    
    def forward(self,x):
        stream1 = self.encoder(x)
        
        
        stream2 = F.relu(self.fc_input(x))
        stream2 = F.relu(self.fc1(stream2))
        stream2 = self.dropout(stream2)
        stream2 = F.relu(self.fc2(stream2))
        
        combined = torch.cat((stream1, stream2), -1)
        outputs = torch.softmax(self.fc3(combined), dim=-1)

        return outputs

In [29]:
batch_size = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

combined_model = combined_net(encoder=trained_encoder_model, input_dim=pbmc.shape[1], device=device)
print(combined_model)

Using cpu device
combined_net(
  (encoder): trained_encoder(
    (dropout): Dropout(p=0.3, inplace=False)
    (gcn): SpectralConvolution (32)
    (gcn2): SpectralConvolution (32)
    (encoder_a1): Linear(in_features=3840, out_features=128, bias=True)
    (encoder_a2): Linear(in_features=128, out_features=32, bias=True)
  )
  (fc_input): Linear(in_features=960, out_features=512, bias=True)
  (fc1): Linear(in_features=512, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=32, bias=True)
  (dropout): Dropout(p=0.3, inplace=False)
  (fc3): Linear(in_features=64, out_features=11, bias=True)
)


In [30]:
loss_fn = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(combined_model.parameters(),
#                              lr = 1e-3,
#                              weight_decay = 1e-8)

optimizer = torch.optim.SGD(combined_model.parameters(),
                             lr = 1e-2)

x_tensor = torch.tensor(pbmc[:len(pbmc)-23].astype(np.float32))
y_tensor = torch.tensor(labels_df[:len(pbmc)-23].values.astype(np.float32))

dataset = TensorDataset(x_tensor, y_tensor)

# batch_size = 32
validation_split = .2
shuffle_dataset = True
random_seed= 30

# Creating data indices for training and validation splits:
dataset_size = len(x_tensor)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# Creating PT data samplers and loaders:
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

train_loader = DataLoader(dataset, batch_size=batch_size, 
                                           sampler=train_sampler)
validation_loader = DataLoader(dataset, batch_size=batch_size,
                                                sampler=valid_sampler)

In [31]:
def train(dataloader, model, loss_fn, optimizer):
    size = math.floor(len(dataloader.dataset)*0.8)
    model.train()
    correct = 0

    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error 
        pred = model(X)
        loss = loss_fn(pred, y)

        correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            correct /= 100*batch_size

            print(f"Train accuracy: {(100*correct):>0.1f}%, loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            correct = 0
            
def test(dataloader, model, loss_fn):
    size = math.floor(len(dataloader.dataset)*0.2)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y.argmax(1)).type(torch.float).sum().item()
    test_loss /= num_batches
    print(correct)
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, combined_model, loss_fn, optimizer)
    test(validation_loader, combined_model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
Train accuracy: 0.0%, loss: 2.429672  [    0/52736]
Train accuracy: 28.0%, loss: 2.324325  [ 3200/52736]
Train accuracy: 30.8%, loss: 2.199360  [ 6400/52736]
Train accuracy: 30.2%, loss: 2.198833  [ 9600/52736]
Train accuracy: 29.6%, loss: 2.199478  [12800/52736]
Train accuracy: 31.3%, loss: 2.106249  [16000/52736]
Train accuracy: 30.4%, loss: 2.261825  [19200/52736]
Train accuracy: 31.6%, loss: 2.199147  [22400/52736]
Train accuracy: 32.0%, loss: 2.261720  [25600/52736]
Train accuracy: 31.8%, loss: 2.355536  [28800/52736]
Train accuracy: 31.2%, loss: 2.230384  [32000/52736]
Train accuracy: 31.5%, loss: 2.230618  [35200/52736]
Train accuracy: 30.9%, loss: 2.324711  [38400/52736]
Train accuracy: 30.0%, loss: 2.230531  [41600/52736]
Train accuracy: 31.6%, loss: 2.261852  [44800/52736]
Train accuracy: 30.7%, loss: 2.261855  [48000/52736]
Train accuracy: 30.5%, loss: 2.261818  [51200/52736]
3990.0
Test Error: 
 Accuracy: 30.3%, Avg loss: 2.240402 

E

In [44]:
for name, param in combined_model.named_parameters():
    print(name, param.max(), param.var())

In [ ]:
for name, param in combined_model.named_parameters():
    print(name, param.shape)

In [89]:
#for standard gcn

trained_model_dict = model.state_dict()

In [59]:
#skeleton for loading trained encoder

class trained_encoder_att(nn.Module):
    
    def __init__(self, input_dim, nhidden, p, adj, batch_size, dropout):
        super(trained_encoder_att, self).__init__()
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.nhidden = nhidden
        self.dropout = nn.Dropout(dropout)
        self.poolsize = 8
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.p = p
        self.adj = Parameter(torch.from_numpy(adj).contiguous().float(), requires_grad=False).to(self.device)
        self.gcn = GraphAttConvolution(self.input_dim, 1, self.nhidden, self.batch_size, bias=False)
#         self.gcn2 = GraphConvolution(self.input_dim, self.nhidden, self.nhidden//2, self.batch_size, bias=False)
        self.encoder_a1 = nn.Linear(int(((self.input_dim//self.p)*nhidden)), 128, bias=True)
        self.encoder_a2 = nn.Linear(128, 32, bias=True)
        
        #Make sure gene order in adj is same!
        #how to implement the attention layer?
        #what's the diff between Parameter and Variable
        
    def graph_max_pool(self, x, p):
        if p > 1:
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            return x
        else:
            return x
        
    def forward(self, x):
        #encoder
        stream1 = torch.unsqueeze(x, -1)
        stream1 = F.relu(self.gcn(stream1, self.adj))
        stream1 = self.graph_max_pool(stream1, self.p)
        stream1 = torch.flatten(stream1, start_dim = 1)
        stream1 = F.relu(self.encoder_a1(stream1))
        outputs = F.relu(self.encoder_a2(stream1))
        
        return outputs

In [90]:
#skeleton for loading trained encoder

class trained_encoder_gcn(nn.Module):
    
    def __init__(self, input_dim, nhidden, p, adj, batch_size, dropout):
        super(trained_encoder_gcn, self).__init__()
        self.batch_size = batch_size
        self.input_dim = input_dim
        self.nhidden = nhidden
        self.dropout = nn.Dropout(dropout)
        self.poolsize = 8
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.p = p
        self.adj = Parameter(torch.from_numpy(adj).contiguous().float(), requires_grad=False).to(self.device)
        self.gcn = GraphConvolution(self.input_dim, 1, self.nhidden, self.batch_size, bias=False)
#         self.gcn2 = GraphConvolution(self.input_dim, self.nhidden, self.nhidden//2, self.batch_size, bias=False)
        self.encoder_a1 = nn.Linear(int(((self.input_dim//self.p)*nhidden)), 128, bias=True)
        self.encoder_a2 = nn.Linear(128, 32, bias=True)
        
        #Make sure gene order in adj is same!
        #how to implement the attention layer?
        #what's the diff between Parameter and Variable
        
    def graph_max_pool(self, x, p):
        if p > 1:
            x = x.permute(0,2,1).contiguous()  # x = B x F x V
            x = nn.MaxPool1d(p)(x)             # B x F x V/p
            x = x.permute(0,2,1).contiguous()  # x = B x V/p x F
            return x
        else:
            return x
        
    def forward(self, x):
        #encoder
        stream1 = torch.unsqueeze(x, -1)
        stream1 = F.relu(self.gcn(stream1, self.adj))
        stream1 = self.graph_max_pool(stream1, self.p)
        stream1 = torch.flatten(stream1, start_dim = 1)
        stream1 = F.relu(self.encoder_a1(stream1))
        outputs = F.relu(self.encoder_a2(stream1))
        
        return outputs

In [91]:
#Trained encoder

batch_size = 32

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

trained_encoder_model = trained_encoder_gcn(pbmc.shape[1], nhidden=32, p=8, adj=L, batch_size=batch_size, dropout=0.3)
# print(model)

Using cpu device


In [92]:
trained_encoder_model_dict = trained_encoder_model.state_dict()

# 1. filter out unnecessary keys
pretrained_dict = {k: v for k, v in trained_model_dict.items() if k in trained_encoder_model_dict}
# 2. overwrite entries in the existing state dict
trained_encoder_model_dict.update(pretrained_dict) 
# # 3. load the new state dict
trained_encoder_model.load_state_dict(pretrained_dict)

<All keys matched successfully>

In [245]:
for k,v in trained_model_dict.items():
    print(k)

fc_input.weight
fc_input.bias
fc1.weight
fc1.bias
fc2.weight
fc2.bias
fc3.weight
fc3.bias


In [246]:
for k,v in trained_encoder_model_dict.items():
    print(k)

adj
gcn.weight
encoder_a1.weight
encoder_a1.bias
encoder_a2.weight
encoder_a2.bias


In [63]:
for param in trained_encoder_model.parameters():
    param.requires_grad = False